In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import folium
%matplotlib notebook

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive

Mounted at /gdrive
/gdrive


In [ ]:
bike_ride_2018 = pd.read_csv('./BA-bikes-rides-2018.csv')

In [ ]:
# put -1 in NaN values
bike_ride_2018 = bike_ride_2018.fillna(-1)
# rename and convert to int some columns 
bike_ride_2018.rename(columns={'fecha_origen_recorrido':'dateTime'}, inplace=True)
bike_ride_2018['id_estacion_origen'] = bike_ride_2018['id_estacion_origen'].astype(int)
bike_ride_2018['id_estacion_destino'] = bike_ride_2018['id_estacion_destino'].astype(int)
# create new index type date
bike_ride_2018['date'] = pd.to_datetime(bike_ride_2018['dateTime']).dt.date
bike_ride_2018.set_index('date', inplace=True)
bike_ride_2018.head()

,id_usuario,genero_usuario,dateTime,id_estacion_origen,nombre_estacion_origen,long_estacion_origen,lat_estacion_origen,domicilio_estacion_origen,duracion_recorrido,fecha_destino_recorrido,id_estacion_destino,nombre_estacion_destino,long_estacion_destino,lat_estacion_destino,domicilio_estacion_destino
date,,,,,,,,,,,,,,,
2018-01-01,5453,M,2018-01-01 00:08:05,45,Uruguay,-58.386913,-34.601616,Tucumán y Uruguay,0 days 00:19:53.000000000,2018-01-01 00:27:58,183,Virrey Cevallos,-58.389692,-34.615638,México 1652 entre Virrey Cevallos y Solís
2018-01-01,673,M,2018-01-01 00:18:05,189,Posadas,-58.385586,-34.588475,Posadas 1350 entre Montevideo y Rodríguez Peña,0 days 00:26:19.000000000,2018-01-01 00:44:24,110,Guardia Vieja,-58.415459,-34.601761,Guardia Vieja 3606 entre Billinghurst y Mario ...
2018-01-01,179119,F,2018-01-01 00:20:14,50,Hospital Rivadavia,-58.401148,-34.583677,Lucena Pereyra 2516 entre Las Heras y Pagano,0 days 00:27:39.000000000,2018-01-01 00:47:53,31,Padilla,-58.439709,-34.603162,Padilla y Warnes Av
2018-01-01,400147,M,2018-01-01 00:20:22,111,Macacha Güemes,-58.363000,-34.604969,Macacha Guemes y Juana Manso,0 days 00:48:51.000000000,2018-01-01 01:09:13,54,Acuña de Figueroa,-58.421705,-34.598222,Lavalle y Acuña de Figueroa
2018-01-01,400156,F,2018-01-01 00:20:31,111,Macacha Güemes,-58.363000,-34.604969,Macacha Guemes y Juana Manso,0 days 00:49:27.000000000,2018-01-01 01:09:58,54,Acuña de Figueroa,-58.421705,-34.598222,Lavalle y Acuña de Figueroa


In [ ]:
# create new df for only a month of bike_ride_2018, to process my tests faster
start_date = '2018-01-01'
end_date = '2018-01-31'
mask  =  ( bike_ride_2018[ 'dateTime' ]  >= start_date )  &  ( bike_ride_2018[ 'dateTime' ]  <=  end_date )
br_2018_net = bike_ride_2018.loc[mask]
#reset the index to default (int)
br_2018_net.reset_index(drop=True, inplace=True)
#print(f"Rows: {br_2018_net.size}")
br_2018_net[['id_usuario', 'id_estacion_origen', 'id_estacion_destino', 'nombre_estacion_origen', 'long_estacion_origen', 'lat_estacion_origen']]

,id_usuario,id_estacion_origen,id_estacion_destino,nombre_estacion_origen,long_estacion_origen,lat_estacion_origen
0,5453,45,183,Uruguay,-58.386913,-34.601616
1,673,189,110,Posadas,-58.385586,-34.588475
2,179119,50,31,Hospital Rivadavia,-58.401148,-34.583677
3,400147,111,54,Macacha Güemes,-58.363000,-34.604969
4,400156,111,54,Macacha Güemes,-58.363000,-34.604969
...,...,...,...,...,...,...
113885,489800,121,187,Yatay,-58.428268,-34.600818
113886,439087,111,152,Macacha Güemes,-58.363000,-34.604969
113887,84701,43,195,Plaza Houssay,-58.398521,-34.599190
113888,5612,17,13,Plaza Almagro,-58.418832,-34.606399


In [ ]:
# Drop all the stations that have -1 in your id
br_2018_net = br_2018_net.drop(br_2018_net[(br_2018_net['id_estacion_origen'] == -1) | (br_2018_net['id_estacion_destino'] == -1)].index)
br_2018_net[['id_usuario', 'id_estacion_origen', 'id_estacion_destino', 'nombre_estacion_origen', 'long_estacion_origen', 'lat_estacion_origen']]

,id_usuario,id_estacion_origen,id_estacion_destino,nombre_estacion_origen,long_estacion_origen,lat_estacion_origen
0,5453,45,183,Uruguay,-58.386913,-34.601616
1,673,189,110,Posadas,-58.385586,-34.588475
2,179119,50,31,Hospital Rivadavia,-58.401148,-34.583677
3,400147,111,54,Macacha Güemes,-58.363000,-34.604969
4,400156,111,54,Macacha Güemes,-58.363000,-34.604969
...,...,...,...,...,...,...
113885,489800,121,187,Yatay,-58.428268,-34.600818
113886,439087,111,152,Macacha Güemes,-58.363000,-34.604969
113887,84701,43,195,Plaza Houssay,-58.398521,-34.599190
113888,5612,17,13,Plaza Almagro,-58.418832,-34.606399


In [ ]:
# create new df only with origin bike stations
origin_stations = br_2018_net[['id_estacion_origen','nombre_estacion_origen','long_estacion_origen', 'lat_estacion_origen']].drop_duplicates().reset_index()
origin_stations.rename(columns={'id_estacion_origen':'station_id_o', 'nombre_estacion_origen':'station_o', 'long_estacion_origen':'long_o', 'lat_estacion_origen':'lat_o'}, inplace=True)
origin_stations.drop(['index'], axis = 1, inplace=True)
origin_stations

,station_id_o,station_o,long_o,lat_o
0,45,Uruguay,-58.386913,-34.601616
1,189,Posadas,-58.385586,-34.588475
2,50,Hospital Rivadavia,-58.401148,-34.583677
3,111,Macacha Güemes,-58.363000,-34.604969
4,121,Yatay,-58.428268,-34.600818
...,...,...,...,...
191,169,Facultad de Psicología,-58.411925,-34.612020
192,124,Ugarteche,-58.412314,-34.581082
193,84,Lavalle,-58.395472,-34.602290
194,62,Córdoba,-58.376901,-34.598516


In [ ]:
# create new df only with destination bike stations
destination_stations = br_2018_net[['id_estacion_destino','nombre_estacion_destino','long_estacion_destino', 'lat_estacion_destino']].drop_duplicates().reset_index()
destination_stations.rename(columns={'id_estacion_destino':'station_id_d', 'nombre_estacion_destino':'station_d', 'long_estacion_destino':'long_d', 'lat_estacion_destino':'lat_d'}, inplace=True)
destination_stations.drop(['index'], axis = 1, inplace=True)
destination_stations

,station_id_d,station_d,long_d,lat_d
0,183,Virrey Cevallos,-58.389692,-34.615638
1,110,Guardia Vieja,-58.415459,-34.601761
2,31,Padilla,-58.439709,-34.603162
3,54,Acuña de Figueroa,-58.421705,-34.598222
4,143,Billinghurst y Valentín Gomez,-58.415759,-34.604635
...,...,...,...,...
191,53,Ricardo Rojas,-58.374282,-34.595881
192,84,Lavalle,-58.395472,-34.602290
193,169,Facultad de Psicología,-58.411925,-34.612020
194,124,Ugarteche,-58.412314,-34.581082


In [ ]:
station_id = br_2018_net.id_estacion_origen.unique()
station_id.sort()
station_id

array([ -1,   1,   2,   3,   4,   5,   6,   7,   8,   9,  11,  12,  13,
        14,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,
        55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,
        68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,
        81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
        94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106,
       107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119,
       120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132,
       133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145,
       146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158,
       160, 161, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173,
       174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 18

In [ ]:
# calculate trips between origin and destination
count_viajes = br_2018_net.groupby(['id_estacion_origen', 'id_estacion_destino']).size()
br_2018_sum_viajes = count_viajes.to_frame(name = 'travels').reset_index()
br_2018_sum_viajes

,id_estacion_origen,id_estacion_destino,travels
0,1,1,166
1,1,2,24
2,1,3,6
3,1,5,27
4,1,6,2
...,...,...,...
23290,200,193,13
23291,200,194,1
23292,200,196,1
23293,200,197,2


In [ ]:
# Merge with origin stations data, then with destination stations data
br_stations_travels_partial = pd.merge(left=br_2018_sum_viajes, right=origin_stations, how='left', left_on='id_estacion_origen', right_on='station_id_o')
br_stations_travels = pd.merge(left=br_stations_travels_partial, right=destination_stations, how='left', left_on='id_estacion_destino', right_on='station_id_d')
br_stations_travels

,id_estacion_origen,id_estacion_destino,travels,station_id_o,station_o,long_o,lat_o,station_id_d,station_d,long_d,lat_d
0,1,1,166,1,Facultad de Derecho,-58.392452,-34.583133,1,Facultad de Derecho,-58.392452,-34.583133
1,1,2,24,1,Facultad de Derecho,-58.392452,-34.583133,2,Retiro,-58.374822,-34.592589
2,1,3,6,1,Facultad de Derecho,-58.392452,-34.583133,3,Aduana,-58.368918,-34.611242
3,1,5,27,1,Facultad de Derecho,-58.392452,-34.583133,5,Plaza Italia,-58.420997,-34.580127
4,1,6,2,1,Facultad de Derecho,-58.392452,-34.583133,6,Parque Lezama,-58.369709,-34.628301
...,...,...,...,...,...,...,...,...,...,...,...
23290,200,193,13,200,Austria y French,-58.404361,-34.588191,193,Arenales y Agüero,-58.405974,-34.591111
23291,200,194,1,200,Austria y French,-58.404361,-34.588191,194,Perón y Francisco Acuña de Figueroa,-58.422572,-34.605952
23292,200,196,1,200,Austria y French,-58.404361,-34.588191,196,Hospital Argerich,-58.366019,-34.627776
23293,200,197,2,200,Austria y French,-58.404361,-34.588191,197,Quintino Bocayuva,-58.423481,-34.613596


In [ ]:
from folium import plugins
# Get a basic world map.
bike_rides_map = folium.Map(location=[-34.59, -58.39], tiles="Stamen Toner", zoom_start=15)
# Add measure tool
measure = folium.plugins.MeasureControl()
measure.add_to(bike_rides_map)
bike_rides_map

In [ ]:
# only take the first 20 rows for display purposes
br_stations_travels = br_stations_travels.iloc[1:20]

# iterate over df 
for index, row in br_stations_travels.iterrows():

  # add origins
  folium.Circle(
    location=[row["lat_o"], row["long_o"]],
    radius=40,    
    color="green",
    fill=True,
    opacity=4,
    tooltip=str(row["station_id_o"])+"-"+row["station_o"],  
  ).add_to(bike_rides_map) 

  # add destinations
  folium.Circle(
    location=[row["lat_d"], row["long_d"]],
    radius=50,
    tooltip=str(row["station_id_d"])+"-"+row["station_d"],
    color="#1300ca",
    fill=True,    
  ).add_to(bike_rides_map) 

  #add travel line origin-destination
  travel_coord = [ [row["lat_o"], row["long_o"]], [row["lat_d"], row["long_d"]] ]
  travel_line = folium.PolyLine(travel_coord, color="orange", weight=2, opacity=1,)
  travel_line.add_to(bike_rides_map)
  
  #add text to polyline
  attr = {'fill': 'black', 'font-weight': 'bold', 'font-size': '14'}
  text_in_line = " " * 30 + "Travels: "+str(row["travels"])+"--->"
  folium.plugins.PolyLineTextPath(travel_line, text_in_line,
                           offset=7, attributes=attr).add_to(bike_rides_map)
bike_rides_map